# IMPORTING LIBRARIES

In [2]:
import pandas as pd
import numpy as np
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC  
from sklearn.decomposition import PCA


# READING CSV FILE

In [3]:
df = pd.read_csv("./creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# UNDERSAMPLING THE DATA

In [4]:
no_frauds = len(df[df['Class'] == 1])
non_fraud_indices = df[df.Class == 0].index
non_fraud_indices = df[df.Class == 0].index
random_indices = np.random.choice(non_fraud_indices, no_frauds, replace=False)
fraud_indices = df[df.Class == 1].index
under_sample_indices = np.concatenate([fraud_indices,random_indices])
under_sample = df.loc[under_sample_indices]

# UNDERSAMPLED DATASET

In [5]:
under_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
6331,7526.0,0.008430,4.137837,-6.240697,6.675732,0.768307,-3.353060,-1.631735,0.154612,-2.795892,...,0.364514,-0.608057,-0.539528,0.128940,1.488481,0.507963,0.735822,0.513574,1.00,1
6334,7535.0,0.026779,4.132464,-6.560600,6.348557,1.329666,-2.513479,-1.689102,0.303253,-3.139409,...,0.370509,-0.576752,-0.669605,-0.759908,1.605056,0.540675,0.737040,0.496699,1.00,1
6336,7543.0,0.329594,3.712889,-5.775935,6.078266,1.667359,-2.420168,-0.812891,0.133080,-2.214311,...,0.156617,-0.652450,-0.551572,-0.716522,1.415717,0.555265,0.530507,0.404474,1.00,1
6338,7551.0,0.316459,3.809076,-5.615159,6.047445,1.554026,-2.651353,-0.746579,0.055586,-2.678679,...,0.208828,-0.511747,-0.583813,-0.219845,1.474753,0.491192,0.518868,0.402528,1.00,1
6427,7610.0,0.725646,2.300894,-5.329976,4.007683,-1.730411,-1.732193,-3.968593,1.063728,-0.486097,...,0.589669,0.109541,0.601045,-0.364700,-1.843078,0.351909,0.594550,0.099372,1.00,1


# SPLITTING THE DATASET INTO TRAIN AND TEST

In [6]:
X_under = under_sample[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19',
                        'V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
y_under = under_sample['Class']
X_under_train, X_under_test, y_under_train, y_under_test = train_test_split(X_under,y_under,test_size = 0.3, random_state = 0)

# RFE ON SVM

In [7]:
model = SVC(kernel='linear')
rfe = RFE(model)
fit = rfe.fit(X_under_train,y_under_train)
print("Number of Features: %d"% fit.n_features_)
print("Selected features: %s"% fit.support_)
print("Ranking of features: %s"% fit.ranking_)  

Number of Features: 14
Selected features: [False  True False  True  True False  True  True False  True  True  True
 False  True False  True  True False False False False False  True False
  True False  True False False]
Ranking of features: [ 4  1 14  1  1 10  1  1 15  1  1  1  5  1  3  1  1  7 13 12 11  8  1  9
  1  6  1  2 16]


# MODEL FITTING ON TRAIN DATA

In [8]:
svclassifier = SVC(kernel='linear')  
X = X_under_train[['V4','V6','V8','V9','V10','V11','V12','V14','V16','V17','V18','V21','V22','V23','V27']]
svclassifier.fit(X, y_under_train)
y_pred = svclassifier.predict(X)
print(confusion_matrix(y_under_train,y_pred))  
print(classification_report(y_under_train,y_pred))  
print(accuracy_score(y_under_train, y_pred))


[[339   4]
 [ 33 312]]
             precision    recall  f1-score   support

          0       0.91      0.99      0.95       343
          1       0.99      0.90      0.94       345

avg / total       0.95      0.95      0.95       688

0.9462209302325582


# MODEL FITTING ON TEST DATA

In [9]:
X2 = X_under_test[['V4','V6','V8','V9','V10','V11','V12','V14','V16','V17','V18','V21','V22','V23','V27']]
y_pred = svclassifier.predict(X2)
print(confusion_matrix(y_under_test,y_pred))  
print(classification_report(y_under_test,y_pred))  
print(accuracy_score(y_under_test, y_pred))


[[145   4]
 [ 16 131]]
             precision    recall  f1-score   support

          0       0.90      0.97      0.94       149
          1       0.97      0.89      0.93       147

avg / total       0.94      0.93      0.93       296

0.9324324324324325


# GRIDSEARCH ON TRAIN DATA

In [10]:
# Logistic regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

logregpipe = Pipeline([('scale', StandardScaler()),
                   ('logreg',LogisticRegression(multi_class="multinomial",solver="lbfgs"))])

# Gridsearch to determine the value of C
param_grid = {'logreg__C':np.arange(0.01,100,10)}
logreg_cv = GridSearchCV(logregpipe,param_grid,cv=5,return_train_score=True)
logreg_cv.fit(X,y_under_train)
print(logreg_cv.best_params_)

bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X,y_under_train)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X,y_under_train)

{'logreg__C': 10.01}


0.9462209302325582

# GRIDSEARCH ON TEST DATA

In [11]:
bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X2,y_under_test)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X2,y_under_test)

0.956081081081081

# CROSS VALIDATION ON TRAIN DATA

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation

predicted = cross_validation.cross_val_predict(LogisticRegression(),X,y_under_train, cv=10)
print (metrics.accuracy_score(y_under_train, predicted))

0.936046511627907


C:\Users\sayal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# CROSS VALIDATION ON TEST DATA

In [13]:
predicted1 = cross_validation.cross_val_predict(LogisticRegression(),X2, y_under_test, cv=10)

print (metrics.classification_report(y_under_test, predicted1))
print (metrics.accuracy_score(y_under_test, predicted1))

             precision    recall  f1-score   support

          0       0.92      0.95      0.93       149
          1       0.94      0.91      0.93       147

avg / total       0.93      0.93      0.93       296

0.9290540540540541


# K BEST ON SVM

In [14]:
array = under_sample.values
test = SelectKBest(score_func=f_classif, k=10)
fit = test.fit(X_under_train, y_under_train)

In [15]:
print("scores_:",test.scores_)
print("pvalues_:",test.pvalues_)
print("selected index:",test.get_support(True))
print("after transform:",test.transform(X_under_train)) 

scores_: [1.82273231e+02 2.50407484e+02 3.48339250e+02 6.75688177e+02
 1.08532650e+02 1.38417108e+02 2.25323506e+02 4.67443081e+00
 2.94612681e+02 4.95293789e+02 6.09895990e+02 6.24717094e+02
 2.20928935e-01 9.15064350e+02 9.07914006e-01 3.87729746e+02
 3.18844720e+02 1.90319957e+02 6.09530327e+01 1.63007782e+01
 2.25057636e+01 9.19749844e-01 2.79064715e+00 9.38206817e+00
 2.73693615e+00 1.82046511e-01 1.12577179e+01 2.42282666e+00
 2.35402548e+00]
pvalues_: [5.25127514e-037 2.60037100e-048 3.53570962e-063 3.20389971e-104
 1.07853663e-023 3.08160131e-029 2.99661564e-044 3.09599033e-002
 3.30226950e-055 5.15112700e-083 7.83756883e-097 1.57557592e-098
 6.38482410e-001 2.24690105e-128 3.41004109e-001 9.24453036e-069
 7.42612176e-059 2.18186235e-038 2.19030487e-014 6.01345771e-005
 2.55012066e-006 3.37878542e-001 9.52720675e-002 2.27694045e-003
 9.85100377e-002 6.69754231e-001 8.36602350e-004 1.20040182e-001
 1.25420999e-001]
selected index: [ 1  2  3  8  9 10 11 13 15 16]
after transform:

# MODEL FITTING ON TRAIN DATA

In [16]:
svclassifier = SVC(kernel='linear')  
X = X_under_train[['V2','V3','V4','V9','V10','V11','V12','V14','V16','V17']]
svclassifier.fit(X, y_under_train)
y_pred = svclassifier.predict(X)
print(confusion_matrix(y_under_train,y_pred))  
print(classification_report(y_under_train,y_pred))  
print(accuracy_score(y_under_train, y_pred))


[[338   5]
 [ 38 307]]
             precision    recall  f1-score   support

          0       0.90      0.99      0.94       343
          1       0.98      0.89      0.93       345

avg / total       0.94      0.94      0.94       688

0.9375


# MODEL FITTING ON TEST DATA

In [17]:
X2 = X_under_test[['V2','V3','V4','V9','V10','V11','V12','V14','V16','V17']]
y_pred = svclassifier.predict(X2)
print(confusion_matrix(y_under_test,y_pred))  
print(classification_report(y_under_test,y_pred))  
print(accuracy_score(y_under_test, y_pred))


[[148   1]
 [ 14 133]]
             precision    recall  f1-score   support

          0       0.91      0.99      0.95       149
          1       0.99      0.90      0.95       147

avg / total       0.95      0.95      0.95       296

0.9493243243243243


# GRIDSEARCH ON TRAIN DATA

In [18]:
# Logistic regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

logregpipe = Pipeline([('scale', StandardScaler()),
                   ('logreg',LogisticRegression(multi_class="multinomial",solver="lbfgs"))])

# Gridsearch to determine the value of C
param_grid = {'logreg__C':np.arange(0.01,100,10)}
logreg_cv = GridSearchCV(logregpipe,param_grid,cv=5,return_train_score=True)
logreg_cv.fit(X,y_under_train)
print(logreg_cv.best_params_)

bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X,y_under_train)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X,y_under_train)

{'logreg__C': 70.01}


0.9433139534883721

# GRIDSEARCH ON TEST DATA

In [19]:
bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X2,y_under_test)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X2,y_under_test)

0.9594594594594594

# CROSS VALIDATION ON TRAIN DATA

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation

predicted = cross_validation.cross_val_predict(LogisticRegression(),X,y_under_train, cv=10)
print (metrics.accuracy_score(y_under_train, predicted))

0.9375


# CROSS VALIDATION ON TEST DATA

In [21]:
predicted1 = cross_validation.cross_val_predict(LogisticRegression(),X2, y_under_test, cv=10)

print (metrics.classification_report(y_under_test, predicted1))
print (metrics.accuracy_score(y_under_test, predicted1))

             precision    recall  f1-score   support

          0       0.91      0.96      0.93       149
          1       0.96      0.90      0.93       147

avg / total       0.93      0.93      0.93       296

0.9324324324324325


# PCA ON SVM

In [22]:
pca = PCA(n_components=5)
X_under_train = pca.fit_transform(X_under_train)
X_under_test = pca.transform(X_under_test)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

[9.94439738e-01 3.85079607e-03 4.69598485e-04 4.51920920e-04
 2.12195813e-04]


# MODEL FITTING ON TRAIN & TEST DATA

In [23]:
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_under_train, y_under_train)
y_pred = svclassifier.predict(X_under_test)
print(confusion_matrix(y_under_test,y_pred))  
print(classification_report(y_under_test,y_pred)) 
print(accuracy_score(y_under_test, y_pred))


[[147   2]
 [ 14 133]]
             precision    recall  f1-score   support

          0       0.91      0.99      0.95       149
          1       0.99      0.90      0.94       147

avg / total       0.95      0.95      0.95       296

0.9459459459459459


# GRIDSEARCH ON TEST & TRAIN DATA

In [24]:
# Logistic regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

logregpipe = Pipeline([('scale', StandardScaler()),
                   ('logreg',LogisticRegression(multi_class="multinomial",solver="lbfgs"))])

# Gridsearch to determine the value of C
param_grid = {'logreg__C':np.arange(0.01,100,10)}
logreg_cv = GridSearchCV(logregpipe,param_grid,cv=5,return_train_score=True)
logreg_cv.fit(X_under_train,y_under_train)
print(logreg_cv.best_params_)

bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X_under_train,y_under_train)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X_under_train,y_under_train)

{'logreg__C': 10.01}


0.9418604651162791

# CROSS VALIDATION ON TEST & TRAIN DATA

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation

predicted = cross_validation.cross_val_predict(LogisticRegression(),X_under_train,y_under_train, cv=10)
print (metrics.accuracy_score(y_under_train, predicted))

0.9418604651162791


In [26]:
predicted1 = cross_validation.cross_val_predict(LogisticRegression(),X_under_test, y_under_test, cv=10)

print (metrics.classification_report(y_under_test, predicted1))
print (metrics.accuracy_score(y_under_test, predicted1))

             precision    recall  f1-score   support

          0       0.91      0.99      0.95       149
          1       0.99      0.90      0.94       147

avg / total       0.95      0.94      0.94       296

0.9425675675675675
